In [2]:
!pip install tensorflow
!pip install pandas numpy matplotlib
!pip install pillow

  Using cached tensorflow-2.20.0-cp312-cp312-win_amd64.whl.metadata (4.6 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.9.23-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached protobuf-6.33.0-cp310-abi3-win_amd64.whl.metadata (593 bytes)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached tensorboard-2.20.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached ml_dtypes-0.5.3-cp312-cp312-win_amd64.whl.metadata (9.2 kB)
  Usi

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import os

# Inception V4 모델 구현

이 노트북에서는 운전자 행동 분류를 위한 Inception V4 모델을 구현합니다.

In [ ]:
# Inception V4 모듈 구현

def conv2d_bn(x, filters, kernel_size, strides=1, padding='same', activation='relu', name=None):
    """Convolution + Batch Normalization + Activation"""
    x = layers.Conv2D(filters, kernel_size, strides=strides, padding=padding, 
                      use_bias=False, name=name)(x)
    x = layers.BatchNormalization(scale=False)(x)
    if activation:
        x = layers.Activation(activation)(x)
    return x

def inception_stem(input_tensor):
    """Inception V4의 Stem 모듈"""
    x = conv2d_bn(input_tensor, 32, 3, strides=2, padding='valid')
    x = conv2d_bn(x, 32, 3, padding='valid')
    x = conv2d_bn(x, 64, 3)
    
    branch_0 = layers.MaxPooling2D(3, strides=2, padding='valid')(x)
    branch_1 = conv2d_bn(x, 96, 3, strides=2, padding='valid')
    x = layers.concatenate([branch_0, branch_1], axis=-1)
    
    branch_0 = conv2d_bn(x, 64, 1)
    branch_0 = conv2d_bn(branch_0, 96, 3, padding='valid')
    
    branch_1 = conv2d_bn(x, 64, 1)
    branch_1 = conv2d_bn(branch_1, 64, (7, 1))
    branch_1 = conv2d_bn(branch_1, 64, (1, 7))
    branch_1 = conv2d_bn(branch_1, 96, 3, padding='valid')
    
    x = layers.concatenate([branch_0, branch_1], axis=-1)
    
    branch_0 = conv2d_bn(x, 192, 3, strides=2, padding='valid')
    branch_1 = layers.MaxPooling2D(3, strides=2, padding='valid')(x)
    x = layers.concatenate([branch_0, branch_1], axis=-1)
    
    return x

def inception_a(input_tensor):
    """Inception-A 모듈"""
    branch_0 = conv2d_bn(input_tensor, 96, 1)
    
    branch_1 = conv2d_bn(input_tensor, 64, 1)
    branch_1 = conv2d_bn(branch_1, 96, 3)
    
    branch_2 = conv2d_bn(input_tensor, 64, 1)
    branch_2 = conv2d_bn(branch_2, 96, 3)
    branch_2 = conv2d_bn(branch_2, 96, 3)
    
    branch_3 = layers.AveragePooling2D(3, strides=1, padding='same')(input_tensor)
    branch_3 = conv2d_bn(branch_3, 96, 1)
    
    x = layers.concatenate([branch_0, branch_1, branch_2, branch_3], axis=-1)
    return x

def inception_b(input_tensor):
    """Inception-B 모듈"""
    branch_0 = conv2d_bn(input_tensor, 384, 1)
    
    branch_1 = conv2d_bn(input_tensor, 192, 1)
    branch_1 = conv2d_bn(branch_1, 224, (1, 7))
    branch_1 = conv2d_bn(branch_1, 256, (7, 1))
    
    branch_2 = conv2d_bn(input_tensor, 192, 1)
    branch_2 = conv2d_bn(branch_2, 192, (1, 7))
    branch_2 = conv2d_bn(branch_2, 224, (7, 1))
    branch_2 = conv2d_bn(branch_2, 224, (1, 7))
    branch_2 = conv2d_bn(branch_2, 256, (7, 1))
    
    branch_3 = layers.AveragePooling2D(3, strides=1, padding='same')(input_tensor)
    branch_3 = conv2d_bn(branch_3, 128, 1)
    
    x = layers.concatenate([branch_0, branch_1, branch_2, branch_3], axis=-1)
    return x

def inception_c(input_tensor):
    """Inception-C 모듈"""
    branch_0 = conv2d_bn(input_tensor, 256, 1)
    
    branch_1 = conv2d_bn(input_tensor, 384, 1)
    branch_1_0 = conv2d_bn(branch_1, 256, (1, 3))
    branch_1_1 = conv2d_bn(branch_1, 256, (3, 1))
    branch_1 = layers.concatenate([branch_1_0, branch_1_1], axis=-1)
    
    branch_2 = conv2d_bn(input_tensor, 384, 1)
    branch_2 = conv2d_bn(branch_2, 448, (1, 3))
    branch_2 = conv2d_bn(branch_2, 512, (3, 1))
    branch_2_0 = conv2d_bn(branch_2, 256, (3, 1))
    branch_2_1 = conv2d_bn(branch_2, 256, (1, 3))
    branch_2 = layers.concatenate([branch_2_0, branch_2_1], axis=-1)
    
    branch_3 = layers.AveragePooling2D(3, strides=1, padding='same')(input_tensor)
    branch_3 = conv2d_bn(branch_3, 256, 1)
    
    x = layers.concatenate([branch_0, branch_1, branch_2, branch_3], axis=-1)
    return x

def reduction_a(input_tensor):
    """Reduction-A 모듈 (Inception-A에서 Inception-B로 전환)"""
    branch_0 = conv2d_bn(input_tensor, 384, 3, strides=2, padding='valid')
    
    branch_1 = conv2d_bn(input_tensor, 192, 1)
    branch_1 = conv2d_bn(branch_1, 224, 3)
    branch_1 = conv2d_bn(branch_1, 256, 3, strides=2, padding='valid')
    
    branch_2 = layers.MaxPooling2D(3, strides=2, padding='valid')(input_tensor)
    
    x = layers.concatenate([branch_0, branch_1, branch_2], axis=-1)
    return x

def reduction_b(input_tensor):
    """Reduction-B 모듈 (Inception-B에서 Inception-C로 전환)"""
    branch_0 = conv2d_bn(input_tensor, 192, 1)
    branch_0 = conv2d_bn(branch_0, 192, 3, strides=2, padding='valid')
    
    branch_1 = conv2d_bn(input_tensor, 256, 1)
    branch_1 = conv2d_bn(branch_1, 256, (1, 7))
    branch_1 = conv2d_bn(branch_1, 320, (7, 1))
    branch_1 = conv2d_bn(branch_1, 320, 3, strides=2, padding='valid')
    
    branch_2 = layers.MaxPooling2D(3, strides=2, padding='valid')(input_tensor)
    
    x = layers.concatenate([branch_0, branch_1, branch_2], axis=-1)
    return x

In [ ]:
def create_inception_v4(input_shape=(299, 299, 3), num_classes=10):
    """전체 Inception V4 모델 생성"""
    inputs = layers.Input(shape=input_shape)
    
    # Stem
    x = inception_stem(inputs)
    
    # 4 x Inception-A
    for _ in range(4):
        x = inception_a(x)
    
    # Reduction-A
    x = reduction_a(x)
    
    # 7 x Inception-B
    for _ in range(7):
        x = inception_b(x)
    
    # Reduction-B
    x = reduction_b(x)
    
    # 3 x Inception-C
    for _ in range(3):
        x = inception_c(x)
    
    # Final pooling and prediction
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = models.Model(inputs=inputs, outputs=outputs, name='inception_v4')
    return model

In [ ]:
# 모델 생성
model = create_inception_v4(input_shape=(img_height, img_width, 3), num_classes=num_classes)
model.summary()

In [ ]:
# 데이터 증강 설정
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 20%를 검증 데이터로 사용
)

test_datagen = ImageDataGenerator(rescale=1./255)

# 학습 데이터 로더
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

# 검증 데이터 로더
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

print(f"학습 샘플 수: {train_generator.samples}")
print(f"검증 샘플 수: {validation_generator.samples}")
print(f"클래스: {train_generator.class_indices}")

In [ ]:
# 모델 컴파일
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 콜백 설정
checkpoint = ModelCheckpoint(
    'best_inception_v4_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

callbacks = [checkpoint, early_stopping, reduce_lr]

In [ ]:
# 모델 학습
epochs = 50

history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=callbacks,
    verbose=1
)

In [ ]:
# 학습 결과 시각화
plt.figure(figsize=(14, 5))

# Accuracy 그래프
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Loss 그래프
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# 테스트 데이터 예측 (submission 파일 생성)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=1,
    class_mode=None,
    shuffle=False
)

# 예측
predictions = model.predict(test_generator, verbose=1)

# 제출 파일 생성
test_filenames = test_generator.filenames
test_filenames = [os.path.basename(f) for f in test_filenames]

submission = pd.DataFrame({
    'img': test_filenames,
    'c0': predictions[:, 0],
    'c1': predictions[:, 1],
    'c2': predictions[:, 2],
    'c3': predictions[:, 3],
    'c4': predictions[:, 4],
    'c5': predictions[:, 5],
    'c6': predictions[:, 6],
    'c7': predictions[:, 7],
    'c8': predictions[:, 8],
    'c9': predictions[:, 9]
})

submission.to_csv('inception_v4_submission.csv', index=False)
print("제출 파일이 생성되었습니다: inception_v4_submission.csv")
submission.head()

In [ ]:
# 모델 저장
model.save('inception_v4_final_model.h5')
print("최종 모델이 저장되었습니다: inception_v4_final_model.h5")

In [ ]:
# 데이터 경로 설정
base_dir = r'c:\Users\USER\PycharmProjects\DeepLearning-Term-Proj'
train_dir = os.path.join(base_dir, 'data', 'imgs', 'train')
test_dir = os.path.join(base_dir, 'data', 'imgs', 'test')

# 이미지 파라미터
img_height, img_width = 299, 299  # Inception V4는 299x299 입력 사이즈 사용
batch_size = 32
num_classes = 10  # c0 ~ c9 (10개 클래스)